# Обрабатываем собранные данные 
EDA:
1) очистим дупликаты
2) разабьем вакансии на группы
3) 

In [96]:
import os
from typing import List
from glob import glob
from tqdm import tqdm

import pandas as pd

### Читаем все файлы и избавдяемся от дубликатов

In [98]:
paths = glob("./dags/data/hh/*", recursive=True)

def read_all_csv_files(paths: List[str]) -> pd.DataFrame:
    """
    Читает все CSV-файлы по переданным путям и объединяет их в один DataFrame.

    Args:
        paths (List[str]): Список путей к CSV-файлам.

    Returns:
        pd.DataFrame: Объединённый DataFrame со всех файлов.
    """

    dataframes: List[pd.DataFrame] = []
    abs_paths = [os.path.abspath(path) for path in paths if path.endswith(".csv")]

    if not abs_paths:
        raise FileNotFoundError(f"В директории '{paths}' не найдено CSV-файлов.")

    for path in tqdm(abs_paths, desc='Download'):
        try:
            df = pd.read_csv(path, sep=';')
            df["filename"] = os.path.basename(path)
            dataframes.append(df)
        except Exception as e:
            continue
    if not dataframes:
        raise ValueError("Не удалось прочитать ни один CSV-файл.")

    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df


df = read_all_csv_files(paths)
df.shape

Download: 100%|██████████| 1344/1344 [00:06<00:00, 223.09it/s]


(41591, 15)

In [99]:
df.drop_duplicates('url', inplace=True)
df_dup.drop_duplicates(['description', 'key_skills', 'name'], inplace=True)
df.shape

(5031, 15)

In [100]:
df_dup = df.drop(columns=['salary_from',	'salary_to', 'currency',	'gross', 'employer_accredited_it', 'filename'])
df_dup.shape

(5031, 9)

In [101]:
df_dup.head()

,vacancy_id,name,area,published_at,experience_id,key_skills,employer_name,description,url
0,125684797,Senior Data Analyst,Кипр,24.10.2025,moreThan6,"Python, Clickhouse, A/B тесты, SQL, Apache Air...",Wisebits,Wisebits — международный IT-холдинг с сильной ...,https://hh.ru/vacancy/125684797
1,126832304,Data Scientist,Израиль,22.10.2025,between3And6,"Power BI, Azure Fabric, Data Factory, Synapse ...","Nitka Technologies, Inc.",Nitka Technologies develops software for custo...,https://hh.ru/vacancy/126832304
2,126884932,Product Analyst,Сербия,23.10.2025,between3And6,"SQL, Tableau, Power BI, Анализ данных",Mayflower,Mayflower is a technology company that alters ...,https://hh.ru/vacancy/126884932
3,126884933,Product Analyst,Кипр,23.10.2025,between3And6,"SQL, Tableau, Power BI, Анализ данных",Mayflower,Mayflower is a technology company that alters ...,https://hh.ru/vacancy/126884933
4,126832308,Data Scientist,Черногория,22.10.2025,between3And6,"Power BI, Azure Fabric, Data Factory, Synapse ...","Nitka Technologies, Inc.",Nitka Technologies develops software for custo...,https://hh.ru/vacancy/126832308


In [102]:
df_dup.to_csv('./data/without_dup_v1.csv', index=False)

### Группировываем вакансии

In [ ]:
df_dup.to_csv('./data/without_dup.csv', index=False)

In [ ]:
# dvc remote modify minio_remote endpointurl https://5a384345053a.ngrok-free.app
dvc remote modify minio_remote access_key_id minioadmin
dvc remote modify minio_remote secret_access_key minioadmin
dvc remote modify minio_remote use_ssl false



# https://58bf996bf8b9.ngrok-free.app